In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

2024-08-31 18:00:32.600858: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = (256, 256)

<p>- Wenn du mit sehr großen Datensätzen arbeitest, können diese oft nicht vollständig in den Arbeitsspeicher (RAM oder GPU-Speicher) geladen werden. Durch das Verarbeiten von Daten in Batches kannst du kleinere Teile des Datensatzes nacheinander verarbeiten, ohne den gesamten Datensatz auf einmal laden zu müssen. </p>
<p>- Bei einer BATCH_SIZE von 16 bedeutet dies, dass 16 Bilder gleichzeitig verarbeitet werden, was viel weniger Speicher benötigt als der gesamte Datensatz. </p>

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    batch_size = BATCH_SIZE,
    image_size = IMAGE_SIZE,
    shuffle = True,
    seed = 24
)

In [ ]:
dataset


In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
print(tf.executing_eagerly())

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

In [ ]:
len(dataset)

In [ ]:
for image_batch, label_batch in dataset.take(1):
    plt.imshow(image_batch[0].numpy().astype("uint8"))
    plt.show

In [ ]:
EPOCHS = 50

In [ ]:
tw = [1,29,3,8,3,2,11,56,67]
tw.take(1)

In [ ]:
def get_partitioned_datasets(dataset, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
        if shuffle:
            dataset = dataset.shuffle(shuffle_size, seed=10)

        dataset_len = len(dataset)
    
        train_len = int(train_split*dataset_len)
        train_dataset = dataset.take(train_len)
        dataset = dataset.skip(train_len)

        val_len = int(val_split*dataset_len)
        val_dataset = dataset.take(val_len)
        dataset = dataset.skip(val_len)

        test_len = dataset_len - train_len - val_len
        test_dataset = dataset.take(test_len)

        return train_dataset, val_dataset, test_dataset

In [ ]:
train_dataset, val_dataset, test_dataset = get_partitioned_datasets(dataset)

In [ ]:
len(train_dataset)

In [ ]:
len(val_dataset)


In [ ]:
len(test_dataset)


In [ ]:
train_dataset = train_dataset.shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)


In [ ]:
ONE_IMAGE_SIZE = 256
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(ONE_IMAGE_SIZE, ONE_IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
    layers.RandomContrast(0.2),
    layers.RandomBrightness(0.2)
])

In [ ]:
CHANNELS = 3
input_shape = (BATCH_SIZE, ONE_IMAGE_SIZE, ONE_IMAGE_SIZE, CHANNELS)
n_classes = 10

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    batch_size=BATCH_SIZE,
    validation_data=val_dataset,
    verbose=1,
    epochs=8,
)